In [3]:
from transformers import BertConfig, BertLMHeadModel, Blip2QFormerConfig, Blip2QFormerModel, InstructBlipQFormerConfig, InstructBlipQFormerModel


model = BertLMHeadModel(BertConfig(add_cross_attention=True, is_decoder=True))

In [1]:
import torch
from craft_buddy.models.vit import PreTrainViT


vit = PreTrainViT.from_blip2_vit_ckpt("../ckpt/eva-vit-g/eva_vit_g.pth")
vit = vit.to("cuda:0")
qformer_weights = torch.load("../ckpt/instruct-blip/instruct_blip_vicuna7b_trimmed.pth")['model']
vit.update_output_layernorm(qformer_weights["ln_vision.weight"], qformer_weights["ln_vision.bias"])
vit.to_precision(torch.float16)

/home/dtong/miniconda3/envs/craft_buddy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


not found match head.weight
not found match head.bias


In [1]:
import torch
qformer_weights = torch.load("../ckpt/instruct-blip/instruct_blip_vicuna7b_trimmed.pth")['model']
print("\n".join(qformer_weights.keys()))

query_tokens
ln_vision.weight
ln_vision.bias
Qformer.bert.embeddings.position_ids
Qformer.bert.embeddings.word_embeddings.weight
Qformer.bert.embeddings.position_embeddings.weight
Qformer.bert.embeddings.LayerNorm.weight
Qformer.bert.embeddings.LayerNorm.bias
Qformer.bert.encoder.layer.0.attention.self.query.weight
Qformer.bert.encoder.layer.0.attention.self.query.bias
Qformer.bert.encoder.layer.0.attention.self.key.weight
Qformer.bert.encoder.layer.0.attention.self.key.bias
Qformer.bert.encoder.layer.0.attention.self.value.weight
Qformer.bert.encoder.layer.0.attention.self.value.bias
Qformer.bert.encoder.layer.0.attention.output.dense.weight
Qformer.bert.encoder.layer.0.attention.output.dense.bias
Qformer.bert.encoder.layer.0.attention.output.LayerNorm.weight
Qformer.bert.encoder.layer.0.attention.output.LayerNorm.bias
Qformer.bert.encoder.layer.0.crossattention.self.query.weight
Qformer.bert.encoder.layer.0.crossattention.self.query.bias
Qformer.bert.encoder.layer.0.crossattention.se

In [5]:
qformer_weights["Qformer.bert.encoder.layer.0.attention.self.key.weight"]

tensor([[-0.0017, -0.0236,  0.0267,  ...,  0.0339,  0.0117, -0.0381],
        [-0.0076, -0.0812, -0.0495,  ..., -0.0206, -0.0437,  0.0078],
        [-0.0320,  0.0524, -0.0123,  ..., -0.0093, -0.0020, -0.0257],
        ...,
        [-0.0235,  0.0224, -0.0348,  ...,  0.0006, -0.0197,  0.0503],
        [ 0.0250,  0.0504,  0.0032,  ...,  0.0317,  0.0530, -0.0699],
        [ 0.0099, -0.0385,  0.0243,  ..., -0.0244,  0.0011, -0.0415]])

In [6]:
import torch

hs_0 = torch.load("../demo/vit_output.pt")
hs_1 = torch.load("../demo/vit_output_1.pt").last_hidden_state

torch.max(torch.abs(hs_0 - hs_1))

tensor(474.5895, device='cuda:0', grad_fn=<MaxBackward1>)

In [8]:
import decord
import einops
import torchvision.transforms.functional as F

decord.bridge.set_bridge('torch')

vr = decord.VideoReader(
    "../data/demo.mp4",
    height=224,
    width=224,
)

image = vr[0]
image = torch.unsqueeze(image, 0).float()
image = einops.rearrange(image, "t h w c -> t c h w")
image = image / 255.0
mean = (0.48145466, 0.4578275, 0.40821073)
std = (0.26862954, 0.26130258, 0.27577711)
image = F.normalize(image, mean, std).cuda(0)

In [13]:
with torch.cuda.amp.autocast():
    res = vit(image).last_hidden_state
res

tensor([[[ 0.6866,  0.2140,  0.0063,  ...,  0.6258, -0.1400,  0.4733],
         [-0.1779,  0.1845,  0.6975,  ...,  0.4893, -1.1423,  0.5277],
         [ 0.2454,  1.3660,  1.0810,  ...,  2.1323, -2.0468,  0.6361],
         ...,
         [-0.2272,  1.0518,  0.8312,  ...,  0.4482, -1.0491,  1.4569],
         [ 0.4646,  1.9827,  0.4013,  ...,  1.2008, -1.2442,  1.8142],
         [ 0.4010, -0.5526,  0.1798,  ..., -0.0161, -0.7259,  1.0158]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [2]:
print("'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2675 MiB |   3841 MiB |   6605 MiB |   3929 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      7 MiB |      4 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   2675 MiB |   3841 MiB |   6605 MiB |   3929 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      7 MiB |      4 MiB |\n|---------------------------------------------------------------------------|\n| Requested memory      |   2645 MiB |   3763 MiB |   6493 MiB |   3848 MiB |\n|       from large pool |   2642 MiB |   3760 MiB |   6488 MiB |   3846 MiB |\n|       from small pool |      2 MiB |      2 MiB |      5 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |   4452 MiB |   4452 MiB |   4452 MiB |      0 B   |\n|       from large pool |   4448 MiB |   4448 MiB |   4448 MiB |      0 B   |\n|       from small pool |      4 MiB |      4 MiB |      4 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Non-releasable memory | 229913 KiB | 655647 KiB |   2738 MiB |   2514 MiB |\n|       from large pool | 228681 KiB | 653481 KiB |   2729 MiB |   2506 MiB |\n|       from small pool |   1231 KiB |   2165 KiB |      8 MiB |      7 MiB |\n|---------------------------------------------------------------------------|\n| Allocations           |     961    |     975    |    6242    |    5281    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    5647    |    5045    |\n|---------------------------------------------------------------------------|\n| Active allocs         |     961    |     975    |    6242    |    5281    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    5647    |    5045    |\n|---------------------------------------------------------------------------|\n| GPU reserved segments |     161    |     161    |     161    |       0    |\n|       from large pool |     159    |     159    |     159    |       0    |\n|       from small pool |       2    |       2    |       2    |       0    |\n|---------------------------------------------------------------------------|\n| Non-releasable allocs |     157    |     158    |    4935    |    4778    |\n|       from large pool |     116    |     116    |     213    |      97    |\n|       from small pool |      41    |      42    |    4722    |    4681    |\n|---------------------------------------------------------------------------|\n| Oversize allocations  |       0    |       0    |       0    |       0    |\n|---------------------------------------------------------------------------|\n| Oversize GPU segments |       0    |       0    |       0    |       0    |\n|===========================================================================|\n'")

'|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2675 MiB |   3841 MiB |   6605 MiB |   3929 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |
|       from small pool |      2 MiB |      2 MiB |      7 MiB |      4 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   2675 MiB |   3841 MiB |   6605 MiB |   3929 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB 

In [3]:
print("'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2675 MiB |   3841 MiB |   6602 MiB |   3927 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      4 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   2675 MiB |   3841 MiB |   6602 MiB |   3927 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      4 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| Requested memory      |   2645 MiB |   3763 MiB |   6493 MiB |   3848 MiB |\n|       from large pool |   2642 MiB |   3760 MiB |   6488 MiB |   3846 MiB |\n|       from small pool |      2 MiB |      2 MiB |      4 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |   4452 MiB |   4452 MiB |   4452 MiB |      0 B   |\n|       from large pool |   4448 MiB |   4448 MiB |   4448 MiB |      0 B   |\n|       from small pool |      4 MiB |      4 MiB |      4 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Non-releasable memory | 229913 KiB | 655647 KiB |   2736 MiB |   2511 MiB |\n|       from large pool | 228681 KiB | 653481 KiB |   2729 MiB |   2506 MiB |\n|       from small pool |   1231 KiB |   2165 KiB |      6 MiB |      4 MiB |\n|---------------------------------------------------------------------------|\n| Allocations           |     961    |     975    |    1802    |     841    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    1207    |     605    |\n|---------------------------------------------------------------------------|\n| Active allocs         |     961    |     975    |    1802    |     841    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    1207    |     605    |\n|---------------------------------------------------------------------------|\n| GPU reserved segments |     161    |     161    |     161    |       0    |\n|       from large pool |     159    |     159    |     159    |       0    |\n|       from small pool |       2    |       2    |       2    |       0    |\n|---------------------------------------------------------------------------|\n| Non-releasable allocs |     157    |     158    |     615    |     458    |\n|       from large pool |     116    |     116    |     213    |      97    |\n|       from small pool |      41    |      42    |     402    |     361    |\n|---------------------------------------------------------------------------|\n| Oversize allocations  |       0    |       0    |       0    |       0    |\n|---------------------------------------------------------------------------|\n| Oversize GPU segments |       0    |       0    |       0    |       0    |\n|===========================================================================|\n'")

'|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2675 MiB |   3841 MiB |   6602 MiB |   3927 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |
|       from small pool |      2 MiB |      2 MiB |      4 MiB |      2 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   2675 MiB |   3841 MiB |   6602 MiB |   3927 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB 

In [4]:
print("'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2675 MiB |   3841 MiB |   6603 MiB |   3927 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      5 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   2675 MiB |   3841 MiB |   6603 MiB |   3927 MiB |\n|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |\n|       from small pool |      2 MiB |      2 MiB |      5 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| Requested memory      |   2645 MiB |   3763 MiB |   6493 MiB |   3848 MiB |\n|       from large pool |   2642 MiB |   3760 MiB |   6488 MiB |   3846 MiB |\n|       from small pool |      2 MiB |      2 MiB |      4 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |   4452 MiB |   4452 MiB |   4452 MiB |      0 B   |\n|       from large pool |   4448 MiB |   4448 MiB |   4448 MiB |      0 B   |\n|       from small pool |      4 MiB |      4 MiB |      4 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Non-releasable memory | 229913 KiB | 655647 KiB |   2736 MiB |   2511 MiB |\n|       from large pool | 228681 KiB | 653481 KiB |   2729 MiB |   2506 MiB |\n|       from small pool |   1231 KiB |   2165 KiB |      6 MiB |      5 MiB |\n|---------------------------------------------------------------------------|\n| Allocations           |     961    |     975    |    2216    |    1255    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    1621    |    1019    |\n|---------------------------------------------------------------------------|\n| Active allocs         |     961    |     975    |    2216    |    1255    |\n|       from large pool |     359    |     360    |     595    |     236    |\n|       from small pool |     602    |     616    |    1621    |    1019    |\n|---------------------------------------------------------------------------|\n| GPU reserved segments |     161    |     161    |     161    |       0    |\n|       from large pool |     159    |     159    |     159    |       0    |\n|       from small pool |       2    |       2    |       2    |       0    |\n|---------------------------------------------------------------------------|\n| Non-releasable allocs |     157    |     158    |    1017    |     860    |\n|       from large pool |     116    |     116    |     213    |      97    |\n|       from small pool |      41    |      42    |     804    |     763    |\n|---------------------------------------------------------------------------|\n| Oversize allocations  |       0    |       0    |       0    |       0    |\n|---------------------------------------------------------------------------|\n| Oversize GPU segments |       0    |       0    |       0    |       0    |\n|===========================================================================|\n'")

'|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2675 MiB |   3841 MiB |   6603 MiB |   3927 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB |   3925 MiB |
|       from small pool |      2 MiB |      2 MiB |      5 MiB |      2 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   2675 MiB |   3841 MiB |   6603 MiB |   3927 MiB |
|       from large pool |   2672 MiB |   3838 MiB |   6597 MiB 